In [33]:
#%pip install kagglehub

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
import kagglehub
from kagglehub import KaggleDatasetAdapter
from statsmodels.tsa.seasonal import STL


c:\Users\osmar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Data

In [82]:
dolarbo = pd.read_excel('./DATA/dolarbo_time_series.xlsx', index_col=0)
dolarbo.index = pd.to_datetime(dolarbo.index)

In [83]:
usdt_luis = pd.read_excel('./DATA/usdt_lucho.xlsx')
# Ensure column names are stripped of extra spaces
usdt_luis.columns = usdt_luis.columns.str.strip()
# Combine 'Fecha' and 'hora' columns to create a 'timestamp' column
usdt_luis['timestamp'] = pd.to_datetime(usdt_luis['Fecha'].astype(str) + " " + usdt_luis['hora'].astype(str))
usdt_luis.index = usdt_luis['timestamp']
usdt_luis = usdt_luis.drop(['Fecha', 'hora', 'timestamp'], axis=1)
usdt_luis

,Compra,Venta
timestamp,,
2023-09-21 18:00:00,7.40,7.55
2023-09-22 18:00:00,7.40,7.55
2023-11-18 10:00:00,7.41,7.56
2023-12-12 17:00:00,7.55,7.65
2024-02-22 09:47:00,8.12,8.40
...,...,...
2025-03-11 16:56:00,11.10,11.60
2025-03-12 12:34:00,11.20,11.70
2025-03-12 18:26:00,11.50,12.00


In [84]:
usdtbol = pd.read_excel('./DATA/usdtbol_time_series.xlsx')
usdtbol['timestamp'] = pd.to_datetime(usdtbol['category'], format='%a %b %d %Y')
usdtbol.index = usdtbol['timestamp']
usdtbol = usdtbol.drop(['category', 'timestamp'], axis=1)
#usdtbol['aux'] = 1
#usdtbol['within_day'] = usdtbol.resample('D')['aux'].transform('sum')
usdtbol

,Bs
timestamp,
2024-08-06,12.89
2024-08-06,12.86
2024-08-06,12.82
2024-08-06,12.72
2024-08-06,12.44
...,...
2025-04-14,13.43
2025-04-14,13.42
2025-04-14,13.41


In [85]:
usdtbol_day = usdtbol.resample('D').mean()

In [86]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['compra'], mode='lines', name='Buy', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['venta'], mode='lines', name='Sell', line=dict(color='red')))
#fig.add_trace(go.Scatter(x=usdt_luis.index, y=usdt_luis['Compra'], mode='lines', name='Sell', line=dict(color='cyan')))
#fig.add_trace(go.Scatter(x=usdt_luis.index, y=usdt_luis['Venta'], mode='lines', name='Sell', line=dict(color='green')))
fig.add_trace(go.Scatter(x=usdtbol_day.index, y=usdtbol_day['Bs'], mode='lines', name='usdtbol.com', line=dict(color='green')))
fig.update_layout(
    title='BOB/USDT',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

## 2. Ultra-High Frequency Data

History Exchange Rate USDT/BOB with hourly frequency [(More info)](https://www.kaggle.com/datasets/darlynbravo/history-exchange-rate-usdtbob).

- **price:** exchange rate
- **available:** amount available
- **advertisers_qty:** quantity of advertisers
- **type:** bid (the highest price a buyer will pay) or ask (the lowest price a seller will accept)
- **timestamp:** captured datetime
- **curr_from:** Currency from
- **curr_to:** Currency to
- **source:** source of data

|                              | Bid                                     | Ask                                      |
|------------------------------|-----------------------------------------|------------------------------------------|
| Definition                   | Price to sell an asset                  | Price to buy an asset                    |
| Relationship to market price | Always slightly lower than market price | Always slightly higher than market price |

Factors that affect bid and ask prices: 
- Market liquidity
- Trading volume
- Supply and demand of currencies
- Overall market volatility
- Economic releases
- Bank holidays   


Measuring the bid-ask spread: The smallest whole unit measurement of the bid-ask spread is called a pip. One pip equals 1/100 of 1%, or 0.0001. 

In [87]:
#kaggle_df = pd.read_csv('./kaggle_currency_exchange_rates.csv')
kaggle_df = pd.read_excel('./DATA/kaggle_currency_exchange_rates.xlsx')
kaggle_df.index = kaggle_df['timestamp']
kaggle_df.index = pd.to_datetime(kaggle_df.index)

kaggle_bid = kaggle_df[kaggle_df['type']=="bid"]
kaggle_ask = kaggle_df[kaggle_df['type']=="ask"]

In [88]:
hourly_minute_bid = kaggle_bid['price'].copy().resample('T').mean()
hourly_minute_bid.dropna(axis=0, inplace=True)

hourly_minute_ask = kaggle_ask['price'].copy().resample('T').mean()
hourly_minute_ask.dropna(axis=0, inplace=True)

In [89]:
seconds_bid = kaggle_bid['price'].copy().resample('S').mean()
seconds_bid.dropna(axis=0, inplace=True)
seconds_ask = kaggle_ask['price'].copy().resample('S').mean()
seconds_ask.dropna(axis=0, inplace=True)

In [90]:
seconds_bid.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 634 entries, 2024-07-09 17:43:37 to 2024-09-02 13:30:03
Series name: price
Non-Null Count  Dtype  
--------------  -----  
634 non-null    float64
dtypes: float64(1)
memory usage: 9.9 KB


In [91]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=seconds_bid.index, y=seconds_bid.values, mode='lines', name='Bid Price', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=seconds_ask.index, y=seconds_ask.values, mode='lines', name='Ask Price', line=dict(color='red')))
fig.update_layout(
    title='Bid and Ask Prices Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

In [92]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=hourly_minute_bid.index, y=hourly_minute_bid.values, mode='lines', name='Bid Price', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=hourly_minute_ask.index, y=hourly_minute_ask.values, mode='lines', name='Ask Price', line=dict(color='red')))
fig.update_layout(
    title='Bid and Ask Prices Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

In [93]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['compra'], mode='lines', name='Buy', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['venta'], mode='lines', name='Sell', line=dict(color='red')))
fig.add_trace(go.Scatter(x=hourly_minute_bid.index, y=hourly_minute_bid.values, mode='lines', name='Bid Price', line=dict(color='pink')))
fig.add_trace(go.Scatter(x=hourly_minute_ask.index, y=hourly_minute_ask.values, mode='lines', name='Ask Price', line=dict(color='orange')))
fig.update_layout(
    title='BOB/USDT',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

In [94]:
# Resample the DataFrame
resampled_df = kaggle_df['price'].resample('M').mean()
resampled_df

timestamp
2024-07-31     9.783415
2024-08-31    10.668275
2024-09-30    10.453263
Freq: M, Name: price, dtype: float64

## 3. EPU Index

In [95]:
epu_day = pd.read_csv('./DATA/EPU_index_bolivia.csv', index_col=0)
#epu_day = pd.read_csv('./DATA/EPU_index_bolivia_v2.csv', index_col=0)
epu_day.index = pd.to_datetime(epu_day.index)
epu_day = epu_day[epu_day.index > '2020-05-03']
epu_day.replace(0, np.nan, inplace=True)
epu_day = epu_day.interpolate(method='linear', limit_direction='forward')
epu_day.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1650 entries, 2020-05-04 to 2025-04-11
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   EPU_index       1650 non-null   float64
 1   EPU_index_norm  1650 non-null   float64
dtypes: float64(2)
memory usage: 38.7 KB


In [96]:
epu_day['7mean_epu'] = epu_day['EPU_index_norm'].rolling(window=7).mean()
epu_day['15mean_epu'] = epu_day['EPU_index_norm'].rolling(window=15).mean()
epu_day['30mean_epu'] = epu_day['EPU_index_norm'].rolling(window=30).mean()

epu_day['7median_epu'] = epu_day['EPU_index_norm'].rolling(window=7).median()
epu_day['15median_epu'] = epu_day['EPU_index_norm'].rolling(window=15).median()
epu_day['30median_epu'] = epu_day['EPU_index_norm'].rolling(window=30).median()

epu_day['7max_epu'] = epu_day['EPU_index_norm'].rolling(window=7).max()
epu_day['15max_epu'] = epu_day['EPU_index_norm'].rolling(window=15).max()
epu_day['30max_epu'] = epu_day['EPU_index_norm'].rolling(window=30).max()

epu_day['7sum_epu'] = epu_day['EPU_index_norm'].rolling(window=7).sum() 
epu_day['15sum_epu'] = epu_day['EPU_index_norm'].rolling(window=15).sum()
epu_day['30sum_epu'] = epu_day['EPU_index_norm'].rolling(window=30).sum() 

In [97]:
epu_stl_7 = STL(epu_day['EPU_index_norm'], period=365, seasonal=7)
epu_stl_fit_7 = epu_stl_7.fit()
epu_day['epu_trend_7'] = epu_stl_fit_7.trend


epu_stl_15 = STL(epu_day['EPU_index_norm'], period=365, seasonal=15)
epu_stl_fit_15 = epu_stl_15.fit()
epu_day['epu_trend_15'] = epu_stl_fit_15.trend

epu_stl_31 = STL(epu_day['EPU_index_norm'], period=365, seasonal=31)
epu_stl_fit_31 = epu_stl_31.fit()
epu_day['epu_trend_31'] = epu_stl_fit_31.trend

In [98]:
epu_day.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1650 entries, 2020-05-04 to 2025-04-11
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   EPU_index       1650 non-null   float64
 1   EPU_index_norm  1650 non-null   float64
 2   7mean_epu       1644 non-null   float64
 3   15mean_epu      1636 non-null   float64
 4   30mean_epu      1621 non-null   float64
 5   7median_epu     1644 non-null   float64
 6   15median_epu    1636 non-null   float64
 7   30median_epu    1621 non-null   float64
 8   7max_epu        1644 non-null   float64
 9   15max_epu       1636 non-null   float64
 10  30max_epu       1621 non-null   float64
 11  7sum_epu        1644 non-null   float64
 12  15sum_epu       1636 non-null   float64
 13  30sum_epu       1621 non-null   float64
 14  epu_trend_7     1650 non-null   float64
 15  epu_trend_15    1650 non-null   float64
 16  epu_trend_31    1650 non-null   float64
dtypes: float64(17)


In [99]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['EPU_index_norm'], mode='lines', name='EPU normalized', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['7mean_epu'], mode='lines', name='EPU 7-day mean.', line=dict(color='pink')))
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['7median_epu'], mode='lines', name='EPU 7-day median.'))
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['7max_epu'], mode='lines', name='EPU 7-day max.'))
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['epu_trend_7'], mode='lines', name='EPU Trend', line=dict(color='green')))
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['venta'], mode='lines', name='USDT Sell', line=dict(color='red'), yaxis='y2'))
fig.update_layout(
    yaxis2=dict(
        title="Bs/USDT",
        overlaying='y',
        side='right'),
    title='BOB/USDT',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

In [100]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['EPU_index_norm'], mode='lines', name='EPU normalized', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['7sum_epu'], mode='lines', name='EPU 7-day sum.', line=dict(color='pink')))
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['15sum_epu'], mode='lines', name='EPU 15-day sum.'))
fig.add_trace(go.Scatter(x=epu_day.index, y=epu_day['30sum_epu'], mode='lines', name='EPU 30-day sum.'))
fig.add_trace(go.Scatter(x=dolarbo.index, y=dolarbo['venta'], mode='lines', name='USDT Sell', line=dict(color='red'), yaxis='y2'))
fig.update_layout(
    yaxis2=dict(
        title="Bs/USDT",
        overlaying='y',
        side='right'),
    title='BOB/USDT',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Type',
    template='plotly_white'
)

fig.show()

## 4. Data for Daily Inflation Forecast

In [116]:
daily_feats = pd.read_excel('./IPC_forecast/daily_features.xlsx', index_col=0)
daily_feats = daily_feats.ffill().bfill()
daily_feats.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5580 entries, 2010-01-01 to 2025-04-11
Columns: 125 entries, beef_lp to libor
dtypes: float64(125)
memory usage: 5.4 MB


In [117]:
weekly_feats = daily_feats.copy().resample('W').mean()
weekly_feats.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 798 entries, 2010-01-03 to 2025-04-13
Freq: W-SUN
Columns: 125 entries, beef_lp to libor
dtypes: float64(125)
memory usage: 785.5 KB


In [118]:
monthly_feats = daily_feats.copy().resample('M').mean()
monthly_feats.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 184 entries, 2010-01-31 to 2025-04-30
Freq: M
Columns: 125 entries, beef_lp to libor
dtypes: float64(125)
memory usage: 181.1 KB


In [119]:
cpi_raw = pd.read_excel('./IPC_forecast/ipc.xlsx', index_col=0)
cpi_raw.index = cpi_raw.index + pd.offsets.MonthEnd(0)

cpi_raw["lag_1"] = cpi_raw['ipc_all'].shift(1)
cpi_raw["lag_2"] = cpi_raw['ipc_all'].shift(2)
cpi_raw["lag_3"] = cpi_raw['ipc_all'].shift(3)
cpi_raw["lag_6"] = cpi_raw['ipc_all'].shift(6)
cpi_raw["lag_9"] = cpi_raw['ipc_all'].shift(9)
cpi_raw["lag_12"] = cpi_raw['ipc_all'].shift(12)

cpi_raw.dropna(axis=0, inplace=True)
cpi_raw.drop(['ipc'], axis=1, inplace=True)

apr2025_lags = [cpi_raw['ipc_all'][-1], cpi_raw['ipc_all'][-2], cpi_raw['ipc_all'][-3], 
                cpi_raw['ipc_all'][-6], cpi_raw['ipc_all'][-9], cpi_raw['ipc_all'][-12]]
cpi_raw.loc['2025-04-30', 'lag_1':'lag_12'] = apr2025_lags

cpi_raw.tail(14)


,ipc_all,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12
2024-03-31,3.066016,2.511500,1.859315,2.113582,2.849693,2.725485,2.529973
2024-04-30,3.464755,3.066016,2.511500,1.859315,2.079867,2.696147,2.689694
2024-05-31,3.517909,3.464755,3.066016,2.511500,1.600883,3.058845,2.895825
2024-06-30,3.847557,3.517909,3.464755,3.066016,2.113582,2.849693,2.725485
2024-07-31,3.951672,3.847557,3.517909,3.464755,1.859315,2.079867,2.696147
2024-08-31,5.194100,3.951672,3.847557,3.517909,2.511500,1.600883,3.058845
2024-09-30,6.184353,5.194100,3.951672,3.847557,3.066016,2.113582,2.849693
2024-10-31,7.941683,6.184353,5.194100,3.951672,3.464755,1.859315,2.079867
2024-11-30,9.499230,7.941683,6.184353,5.194100,3.517909,2.511500,1.600883
2024-12-31,9.980202,9.499230,7.941683,6.184353,3.847557,3.066016,2.113582


In [120]:
#monthly_lags = cpi_raw.iloc[:,1:].copy()

daily_feats_adj = pd.merge(cpi_raw, daily_feats, left_index=True, right_index=True, how='right')
daily_feats_adj = daily_feats_adj.bfill()
daily_feats_adj.loc['2025-04-01':, 'lag_1':'lag_12'] = apr2025_lags

daily_feats_adj.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5580 entries, 2010-01-01 to 2025-04-11
Columns: 132 entries, ipc_all to libor
dtypes: float64(132)
memory usage: 5.8 MB


In [121]:
monthly_target_feats = pd.merge(cpi_raw, monthly_feats, left_index=True, right_index=True, how='right')
monthly_target_feats.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 184 entries, 2010-01-31 to 2025-04-30
Freq: M
Columns: 132 entries, ipc_all to libor
dtypes: float64(132)
memory usage: 195.3 KB


In [122]:
daily_feats_adj.to_excel('./IPC_forecast/daily_feats_adj.xlsx')
monthly_target_feats.to_excel('./IPC_forecast/monthly_target_feats.xlsx')

daily_feats_adj.to_csv('./IPC_forecast/daily_test_set.csv')
monthly_target_feats.to_csv('./IPC_forecast/monthly_train_val_sets.csv')

## 5. Daily Dataframe

Bs/USDT series

In [139]:
dolarbo_day = dolarbo.copy().resample('D').max()
dolarbo_day.interpolate(method='linear', limit_direction='forward', inplace=True)
dolarbo_day

,compra,venta
timestamp,,
2023-09-21,7.400000,7.550000
2023-09-22,7.400000,7.550000
2023-09-23,7.400175,7.550175
2023-09-24,7.400351,7.550351
2023-09-25,7.400526,7.550526
...,...,...
2025-04-10,13.000000,13.700000
2025-04-11,13.300000,13.800000
2025-04-12,13.200000,13.700000


EPU series

In [140]:
daily_df = epu_day.copy()
daily_df = daily_df.merge(dolarbo_day[['compra', 'venta']], how='left', left_index=True, right_index=True)

daily_df['depre_compra'] = daily_df['compra'] - daily_df['compra'].shift(1)
daily_df['depre_venta'] = daily_df['venta'] - daily_df['venta'].shift(1)

daily_df['g1_compra'] = (daily_df['compra'].pct_change(1)) * 100
daily_df['dg1_venta'] = (daily_df['venta'].pct_change(1)) * 100

Daily Inflation forecast

In [141]:
daily_inf = pd.read_csv('./IPC_forecast/daily_forecast.csv', index_col=0)
daily_inf.index = pd.to_datetime(daily_inf.index)
daily_inf.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5580 entries, 2010-01-01 to 2025-04-11
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   cpi         183 non-null    float64
 1   ridge       5580 non-null   float64
 2   lasso       5580 non-null   float64
 3   ada         5580 non-null   float64
 4   gbr         5580 non-null   float64
 5   rf          5580 non-null   float64
 6   et          5580 non-null   float64
 7   set         5580 non-null   object 
 8   min         5580 non-null   float64
 9   max         5580 non-null   float64
 10  w_avg       5580 non-null   float64
 11  w_avg_best  5580 non-null   float64
 12  forecast    5580 non-null   float64
dtypes: float64(12), object(1)
memory usage: 610.3+ KB


In [142]:
daily_inf

,cpi,ridge,lasso,ada,gbr,rf,et,set,min,max,w_avg,w_avg_best,forecast
2010-01-01,NaN,-0.199693,0.465266,0.073078,0.287553,0.357041,0.330907,day,-0.199693,0.465266,0.119384,0.072162,-0.199693
2010-01-02,NaN,-0.100558,0.488982,0.073078,0.304497,0.421485,0.282262,day,-0.100558,0.488982,0.157433,0.117740,-0.150126
2010-01-03,NaN,-0.067680,0.489608,0.073078,0.304497,0.421485,0.288440,day,-0.067680,0.489608,0.170845,0.137182,-0.122644
2010-01-04,NaN,-0.024177,0.495058,0.126990,0.506786,0.446593,0.336407,day,-0.024177,0.506786,0.230014,0.174163,-0.098027
2010-01-05,NaN,-0.304013,0.494714,0.155257,0.490205,0.393022,0.312822,day,-0.304013,0.494714,0.119140,0.017212,-0.139224
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-07,NaN,14.528062,15.080599,14.632839,13.514357,12.898184,13.089919,day,12.898184,15.080599,14.117849,14.266532,14.425768
2025-04-08,NaN,15.207565,15.111413,14.632839,13.489677,12.859263,13.135451,day,12.859263,15.207565,14.377009,14.650649,14.523492
2025-04-09,NaN,15.208234,15.110842,14.632839,13.453171,12.858693,12.941625,day,12.858693,15.208234,14.337798,14.600616,14.599575
2025-04-10,NaN,15.112431,15.086127,14.632839,13.396325,12.811195,12.912505,day,12.811195,15.112431,14.283011,14.536459,14.650860


In [143]:
daily_df = daily_df.merge(daily_inf[['forecast']], how='left', left_index=True, right_index=True)

In [144]:
daily_df

,EPU_index,EPU_index_norm,7mean_epu,15mean_epu,30mean_epu,7median_epu,15median_epu,30median_epu,7max_epu,15max_epu,...,epu_trend_7,epu_trend_15,epu_trend_31,compra,venta,depre_compra,depre_venta,g1_compra,dg1_venta,forecast
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-05-04,0.006237,154.149913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,263.972681,280.907861,286.452046,NaN,NaN,NaN,NaN,NaN,NaN,1.495791
2020-05-05,0.008230,203.401657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,263.627990,280.466430,285.974095,NaN,NaN,NaN,NaN,NaN,NaN,1.502514
2020-05-09,0.010223,252.653401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,263.283524,280.025350,285.496545,NaN,NaN,NaN,NaN,NaN,NaN,1.508744
2020-05-10,0.012215,301.905145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,262.939282,279.584622,285.019400,NaN,NaN,NaN,NaN,NaN,NaN,1.510701
2020-05-13,0.014208,351.156889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,262.595266,279.144249,284.542661,NaN,NaN,NaN,NaN,NaN,NaN,1.513392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-06,0.016653,411.596524,196.125074,194.996179,260.738654,165.652609,165.652609,212.838363,411.596524,579.802368,...,202.436498,197.020643,194.955636,12.6,13.1,0.0,0.0,0.000000,0.000000,14.408719
2025-04-07,0.006419,158.646676,197.902951,166.919133,260.906015,165.652609,158.646676,212.838363,411.596524,411.596524,...,202.407374,196.952140,194.871211,12.7,13.2,0.1,0.1,0.793651,0.763359,14.425768
2025-04-08,0.012535,309.802680,224.103930,161.498310,254.464835,191.345526,158.646676,212.838363,411.596524,411.596524,...,202.378162,196.883488,194.786608,12.8,13.3,0.1,0.1,0.787402,0.757576,14.523492


In [145]:
daily_df.to_excel('./daily_df.xlsx')

In [147]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_df.index, y=daily_df['30mean_epu'], mode='lines', name='EPU 30-day avg.'))
fig.add_trace(go.Scatter(x=daily_df.index, y=daily_df['30max_epu'], mode='lines', name='EPU 30-day max.'))
fig.add_trace(go.Scatter(x=daily_df.index, y=daily_df['compra'], mode='lines', name='Bs/USDT', line=dict(color='black'), yaxis='y2'))
fig.add_trace(go.Scatter(x=daily_df.index, y=daily_df['forecast'], mode='lines', name='y-o-y Inflation', line=dict(color='green'), yaxis='y2'))
fig.update_layout(
    yaxis2=dict(
        title="Bs/USDT and % (Inflation)",
        overlaying='y',
        side='right'),
    #title='BOB/USDT',
    xaxis_title='Timestamp',
    yaxis_title='',
    #legend_title='Type',
    template='plotly_white'
)
fig.show()


In [152]:
aux_day = daily_df.copy()
aux_day = aux_day.dropna(axis=0)

daily_correlation_matrix = aux_day.corr()
daily_correlation_matrix

,EPU_index,EPU_index_norm,7mean_epu,15mean_epu,30mean_epu,7median_epu,15median_epu,30median_epu,7max_epu,15max_epu,...,epu_trend_7,epu_trend_15,epu_trend_31,compra,venta,depre_compra,depre_venta,g1_compra,dg1_venta,forecast
EPU_index,1.000000,1.000000,0.532742,0.428152,0.315731,0.482937,0.407516,0.258227,0.448445,0.309513,...,-0.005086,-0.050177,-0.054271,0.091479,0.085631,0.066261,0.055480,0.068342,0.058811,0.053093
EPU_index_norm,1.000000,1.000000,0.532742,0.428152,0.315731,0.482937,0.407516,0.258227,0.448445,0.309513,...,-0.005086,-0.050177,-0.054271,0.091479,0.085631,0.066261,0.055480,0.068342,0.058811,0.053093
7mean_epu,0.532742,0.532742,1.000000,0.827470,0.601329,0.922111,0.789938,0.521019,0.836223,0.614781,...,0.008742,-0.065440,-0.072493,0.146924,0.138937,0.018048,0.022129,0.021452,0.029155,0.084971
15mean_epu,0.428152,0.428152,0.827470,1.000000,0.806063,0.793858,0.946332,0.754090,0.669385,0.756559,...,0.041666,-0.055099,-0.065524,0.204191,0.196814,0.038578,0.046917,0.041741,0.053417,0.120550
30mean_epu,0.315731,0.315731,0.601329,0.806063,1.000000,0.615225,0.773146,0.954593,0.445179,0.539987,...,0.123139,-0.010233,-0.027284,0.305323,0.297641,0.025889,0.034472,0.027692,0.035705,0.179839
7median_epu,0.482937,0.482937,0.922111,0.793858,0.615225,1.000000,0.809208,0.568096,0.604297,0.457541,...,-0.035779,-0.090372,-0.092850,0.074106,0.065417,0.018690,0.022571,0.022332,0.029504,0.011661
15median_epu,0.407516,0.407516,0.789938,0.946332,0.773146,0.809208,1.000000,0.751688,0.562247,0.577447,...,0.000596,-0.079787,-0.086930,0.152429,0.144287,0.027782,0.035489,0.031743,0.042010,0.072080
30median_epu,0.258227,0.258227,0.521019,0.754090,0.954593,0.568096,0.751688,1.000000,0.340838,0.436415,...,0.057047,-0.030416,-0.037291,0.182496,0.171827,0.014973,0.019943,0.016021,0.020482,0.051314
7max_epu,0.448445,0.448445,0.836223,0.669385,0.445179,0.604297,0.562247,0.340838,1.000000,0.703415,...,0.110274,0.027392,0.015558,0.209073,0.205412,0.032546,0.031314,0.033137,0.034765,0.156461
15max_epu,0.309513,0.309513,0.614781,0.756559,0.539987,0.457541,0.577447,0.436415,0.703415,1.000000,...,0.175238,0.071568,0.055282,0.280815,0.278178,0.038891,0.049120,0.038139,0.053188,0.228139


In [153]:
daily_correlation_matrix.to_excel('./daily_correlation_matrix.xlsx')

In [150]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=aux_day.index, y=aux_day['30mean_epu'], mode='lines', name='EPU 30-day avg.'))
#fig.add_trace(go.Scatter(x=aux_day.index, y=aux_day['30max_epu'], mode='lines', name='EPU 30-day max.'))
fig.add_trace(go.Scatter(x=aux_day.index, y=aux_day['compra'], mode='lines', name='Bs/USDT', line=dict(color='black'), yaxis='y2'))
fig.add_trace(go.Scatter(x=aux_day.index, y=aux_day['forecast'], mode='lines', name='y-o-y Inflation', line=dict(color='green'), yaxis='y2'))
fig.update_layout(
    yaxis2=dict(
        title="Bs/USDT and % (Inflation)",
        overlaying='y',
        side='right'),
    #title='BOB/USDT',
    xaxis_title='Timestamp',
    yaxis_title='',
    #legend_title='Type',
    template='plotly_white'
)
fig.show()


## DSGE Data

### Raw data

In [154]:
epu_month = epu_day.resample('M').mean()
epu_month.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2020-05-31 to 2025-04-30
Freq: M
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   EPU_index       60 non-null     float64
 1   EPU_index_norm  60 non-null     float64
 2   7mean_epu       60 non-null     float64
 3   15mean_epu      59 non-null     float64
 4   30mean_epu      58 non-null     float64
 5   7median_epu     60 non-null     float64
 6   15median_epu    59 non-null     float64
 7   30median_epu    58 non-null     float64
 8   7max_epu        60 non-null     float64
 9   15max_epu       59 non-null     float64
 10  30max_epu       58 non-null     float64
 11  7sum_epu        60 non-null     float64
 12  15sum_epu       59 non-null     float64
 13  30sum_epu       58 non-null     float64
 14  epu_trend_7     60 non-null     float64
 15  epu_trend_15    60 non-null     float64
 16  epu_trend_31    60 non-null     float64
dtypes: float6

In [155]:
usdtbol_month = usdtbol.resample('M').mean()
dolarbo_month = dolarbo.resample('M').mean() 
#usdtbol_month = usdtbol.resample('M').max()
#dolarbo_month = dolarbo.resample('M').max() 

monthly_usdt = pd.merge(usdtbol_month, dolarbo_month, how='outer', left_index=True, right_index=True)
monthly_usdt.columns = ['usdtbol', 'dolarbo_compra', 'dolarbo_venta']
monthly_usdt = monthly_usdt.interpolate(method='linear', limit_direction='forward')

monthly_usdt

,usdtbol,dolarbo_compra,dolarbo_venta
timestamp,,,
2023-09-30,NaN,7.400000,7.550000
2023-10-31,NaN,7.405000,7.555000
2023-11-30,NaN,7.410000,7.560000
2023-12-31,NaN,7.550000,7.650000
2024-01-31,NaN,7.835000,8.027500
2024-02-29,NaN,8.120000,8.405000
2024-03-31,NaN,8.146667,8.246667
2024-04-30,NaN,8.321667,8.471667
2024-05-31,NaN,8.496667,8.696667


In [156]:
igae_ipc = pd.read_excel('./DATA/igae_ipc.xlsx', index_col=0)
igae_ipc.index = igae_ipc.index.to_period('M').to_timestamp('M')

# Perform seasonal decomposition for 'igae'
igae_stl = STL(igae_ipc['igae'], seasonal=13)
igae_decomposition = igae_stl.fit()
igae_seasonally_adjusted = igae_ipc['igae'] - igae_decomposition.seasonal

# Perform seasonal decomposition for 'ipc'
ipc_stl = STL(igae_ipc['ipc'], seasonal=13)
ipc_decomposition = ipc_stl.fit()
ipc_seasonally_adjusted = igae_ipc['ipc'] - ipc_decomposition.seasonal

# Add the seasonally adjusted series to the DataFrame
igae_ipc['igae_sa'] = igae_seasonally_adjusted
igae_ipc['ipc_sa'] = ipc_seasonally_adjusted

# Computing Inflation rate
igae_ipc['inf'] = (igae_ipc['ipc_sa'].pct_change(1)) * 100
igae_ipc['inf_12'] = (igae_ipc['ipc'].pct_change(12)) * 100
# Computing y-o-y GDP growth rate
igae_ipc['g12_y'] = (igae_ipc['igae_sa'].pct_change(12)) * 100
igae_ipc['igae_trend'] = igae_decomposition.trend
igae_ipc['g_gap'] = ((igae_ipc['igae'] / igae_ipc['igae_trend']) - 1) * 100
igae_ipc['g_gap_sa'] = ((igae_ipc['igae_sa'] / igae_ipc['igae_trend']) - 1) * 100

igae_ipc.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 420 entries, 1990-01-31 to 2024-12-31
Freq: M
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   igae        420 non-null    float64
 1   ipc         420 non-null    float64
 2   exchange    402 non-null    float64
 3   igae_sa     420 non-null    float64
 4   ipc_sa      420 non-null    float64
 5   inf         419 non-null    float64
 6   inf_12      408 non-null    float64
 7   g12_y       408 non-null    float64
 8   igae_trend  420 non-null    float64
 9   g_gap       420 non-null    float64
 10  g_gap_sa    420 non-null    float64
dtypes: float64(11)
memory usage: 39.4 KB


### Monthly dataframe

In [157]:
month_raw = igae_ipc.copy()
month_raw = month_raw[month_raw.index >= '2020-05-31']

# Filling Exchange rate NaN values
month_raw.loc['2023-09-30':'2024-12-31', 'exchange'] = monthly_usdt.loc['2023-09-30':'2024-12-31', 'dolarbo_venta']
month_raw['exchange'] = month_raw['exchange'].interpolate(method='linear', limit_direction='forward')

# Merge with EPU data
month_raw = month_raw.merge(epu_month, how='left', left_index=True, right_index=True)

# Percentage deviation from official Exchange rate (Bs6.96)
month_raw['depre'] = month_raw['exchange'] - month_raw['exchange'].shift(1)
month_raw['e_dev'] = (np.log(month_raw['exchange']) - np.log(6.96)) * 100

In [158]:
month_raw.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 56 entries, 2020-05-31 to 2024-12-31
Freq: M
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   igae            56 non-null     float64
 1   ipc             56 non-null     float64
 2   exchange        56 non-null     float64
 3   igae_sa         56 non-null     float64
 4   ipc_sa          56 non-null     float64
 5   inf             56 non-null     float64
 6   inf_12          56 non-null     float64
 7   g12_y           56 non-null     float64
 8   igae_trend      56 non-null     float64
 9   g_gap           56 non-null     float64
 10  g_gap_sa        56 non-null     float64
 11  EPU_index       56 non-null     float64
 12  EPU_index_norm  56 non-null     float64
 13  7mean_epu       56 non-null     float64
 14  15mean_epu      55 non-null     float64
 15  30mean_epu      54 non-null     float64
 16  7median_epu     56 non-null     float64
 17  15median_

In [159]:
aux_month = month_raw['2023-06-30':].copy()
aux_month.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19 entries, 2023-06-30 to 2024-12-31
Freq: M
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   igae            19 non-null     float64
 1   ipc             19 non-null     float64
 2   exchange        19 non-null     float64
 3   igae_sa         19 non-null     float64
 4   ipc_sa          19 non-null     float64
 5   inf             19 non-null     float64
 6   inf_12          19 non-null     float64
 7   g12_y           19 non-null     float64
 8   igae_trend      19 non-null     float64
 9   g_gap           19 non-null     float64
 10  g_gap_sa        19 non-null     float64
 11  EPU_index       19 non-null     float64
 12  EPU_index_norm  19 non-null     float64
 13  7mean_epu       19 non-null     float64
 14  15mean_epu      19 non-null     float64
 15  30mean_epu      19 non-null     float64
 16  7median_epu     19 non-null     float64
 17  15median_

In [160]:
#month_raw_corr = month_raw.corr()
month_raw_corr = aux_month.corr()
#month_raw_corr.to_excel('./month_raw_corr.xlsx')
month_raw_corr

,igae,ipc,exchange,igae_sa,ipc_sa,inf,inf_12,g12_y,igae_trend,g_gap,...,15max_epu,30max_epu,7sum_epu,15sum_epu,30sum_epu,epu_trend_7,epu_trend_15,epu_trend_31,depre,e_dev
igae,1.000000,0.394384,0.227999,-0.062818,0.314493,0.031038,0.386096,-0.058833,0.034804,0.997007,...,-0.203147,-0.144830,-0.324047,-0.328152,-0.307494,0.080640,-0.043631,-0.066089,-0.299550,0.216062
ipc,0.394384,1.000000,0.938883,-0.043318,0.993211,0.781053,0.972011,-0.590116,0.398106,0.362728,...,0.369707,0.450857,0.166932,0.180211,0.245279,0.697646,0.603332,0.577837,0.093290,0.931751
exchange,0.227999,0.938883,1.000000,0.074753,0.948349,0.807125,0.861350,-0.518112,0.525958,0.186692,...,0.498207,0.576033,0.303371,0.318472,0.365027,0.793811,0.721902,0.702719,0.285799,0.998328
igae_sa,-0.062818,-0.043318,0.074753,1.000000,-0.010211,-0.100809,-0.144969,0.605095,0.328218,-0.088014,...,-0.136534,-0.115388,-0.289065,-0.280759,-0.349185,0.371763,0.464813,0.479428,0.318046,0.098915
ipc_sa,0.314493,0.993211,0.948349,-0.010211,1.000000,0.807770,0.962737,-0.609088,0.429512,0.280387,...,0.384108,0.459903,0.173845,0.191858,0.261539,0.743285,0.665724,0.642664,0.109579,0.944459
inf,0.031038,0.781053,0.807125,-0.100809,0.807770,1.000000,0.725585,-0.760080,0.534277,-0.011576,...,0.547602,0.668662,0.490927,0.508404,0.592927,0.710559,0.645902,0.629249,0.127626,0.809834
inf_12,0.386096,0.972011,0.861350,-0.144969,0.962737,0.725585,1.000000,-0.633282,0.208223,0.369055,...,0.250165,0.359134,0.060680,0.080871,0.159094,0.554733,0.466922,0.440540,0.023315,0.845873
g12_y,-0.058833,-0.590116,-0.518112,0.605095,-0.609088,-0.760080,-0.633282,1.000000,-0.193047,-0.042758,...,-0.395532,-0.427402,-0.397886,-0.409713,-0.502993,-0.360039,-0.297659,-0.281470,0.092986,-0.510928
igae_trend,0.034804,0.398106,0.525958,0.328218,0.429512,0.534277,0.208223,-0.193047,1.000000,-0.042512,...,0.513000,0.475511,0.412310,0.421838,0.459729,0.877208,0.828395,0.824129,0.202358,0.569325
g_gap,0.997007,0.362728,0.186692,-0.088014,0.280387,-0.011576,0.369055,-0.042758,-0.042512,1.000000,...,-0.242799,-0.181574,-0.355848,-0.360579,-0.343000,0.012263,-0.108280,-0.130398,-0.314992,0.171424


In [161]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=month_raw.index, y=month_raw['inf'], mode='lines', name='Inflation', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=month_raw.index, y=month_raw['exchange'], mode='lines', name='Exchange', line=dict(color='pink')))
fig.add_trace(go.Scatter(x=month_raw.index, y=month_raw['30max_epu'], mode='lines', name='Sentiment', line=dict(color='green'), yaxis='y2'))
fig.update_layout(
    yaxis2=dict(
        title="Sentiment Index",
        overlaying='y',
        side='right'),
    #title='IPC',
    xaxis_title='',
    yaxis_title='% (Inflation) and Bs/USDT (FX)',
    legend_title='',
    template='plotly_white'
)
fig.show()

In [162]:
month_raw.describe()

,igae,ipc,exchange,igae_sa,ipc_sa,inf,inf_12,g12_y,igae_trend,g_gap,...,15max_epu,30max_epu,7sum_epu,15sum_epu,30sum_epu,epu_trend_7,epu_trend_15,epu_trend_31,depre,e_dev
count,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,...,55.000000,54.000000,56.000000,55.000000,54.000000,56.000000,56.000000,56.000000,55.000000,56.000000
mean,319.527500,108.676250,7.607870,318.587139,108.650728,0.263623,2.276620,1.940027,318.955574,0.148854,...,420.545505,517.145447,1358.391485,2859.837827,5656.949108,189.994534,191.120310,191.393142,0.080073,7.741107
std,27.198231,4.351794,1.280137,17.749712,4.263867,0.418136,2.121523,8.200805,13.670883,7.124104,...,162.052746,192.981605,506.297089,994.196457,1794.758548,28.255427,32.506256,33.834818,0.261827,14.683329
min,230.710000,104.320000,6.960000,244.703063,104.004812,-0.834819,-1.099422,-25.959419,290.095703,-20.575923,...,160.563043,268.327399,609.211726,1444.783970,3028.171890,152.417526,149.977513,148.801003,-0.722891,0.000000
25%,304.540000,105.217500,6.960000,310.159374,105.206906,0.043529,0.890689,-0.133735,309.656537,-4.213073,...,298.354643,358.181241,1014.293668,2218.128231,4371.684697,170.357324,167.013522,166.189188,0.000000,0.000000
50%,321.725000,107.305000,6.960000,321.603045,107.387550,0.244627,1.825834,2.320962,320.923997,-0.409851,...,389.851353,477.000926,1241.553120,2682.806725,5214.290218,186.653462,186.733511,185.575938,0.000000,0.000000
75%,338.190000,110.457500,7.556250,330.970377,110.317437,0.501600,2.936580,4.536099,331.970769,5.511484,...,499.710044,660.046958,1517.876731,3368.516968,6487.129439,205.857375,207.496232,207.655391,0.047500,8.219552
max,371.240000,122.210000,11.398649,345.969693,121.198701,1.241081,9.980202,31.885710,334.371140,16.184632,...,817.701449,1007.567210,2975.241535,6652.523692,11764.086739,262.081033,278.486497,283.830776,1.194706,49.331533


In [163]:
from statsmodels.tsa.stattools import adfuller

# Perform ADF test for each column in bolivia_data
for column in month_raw.columns:
    result = adfuller(month_raw[column].dropna(), maxlag=3)
    print(f"ADF Statistic for {column}: {result[0]}")
    print(f"p-value for {column}: {result[1]}")
    print("Stationary" if result[1] < 0.05 else "Non-stationary")
    print("-" * 50)

ADF Statistic for igae: -4.3400192333675
p-value for igae: 0.00037842730030576965
Stationary
--------------------------------------------------
ADF Statistic for ipc: 6.77863902366668
p-value for ipc: 1.0
Non-stationary
--------------------------------------------------
ADF Statistic for exchange: 2.457493288323336
p-value for exchange: 0.999034847705092
Non-stationary
--------------------------------------------------
ADF Statistic for igae_sa: -5.272235282364404
p-value for igae_sa: 6.249342436087502e-06
Stationary
--------------------------------------------------
ADF Statistic for ipc_sa: 6.256184980741005
p-value for ipc_sa: 1.0
Non-stationary
--------------------------------------------------
ADF Statistic for inf: -2.399828233163783
p-value for inf: 0.1417848944763807
Non-stationary
--------------------------------------------------
ADF Statistic for inf_12: 2.5444721454725525
p-value for inf_12: 0.9990628022021624
Non-stationary
-------------------------------------------------

In [164]:
month_raw.to_excel('./DSGE/bolivia_data.xlsx')